# SHIFT

Python package for creating synthetic power distribution model. This package uses Grid Data Models to represent power distribution components and Ditto for writing case files specific to simulators such as OpenDSS, Cyme, Synergi and others.

Currently, SHIFT uses open street parcel information and road networks to build the distribution model.

For model conversion. We will need to install DiTTo. To do that, use the command below

In [ ]:
pip install nrel-shift --upgrade
pip install networkx~=3.0

Note: you may need to restart the kernel to use updated packages.


We start by loading a sample GDM system using the gdmloader.

In [2]:
from gdm.distribution import DistributionSystem
from gdmloader.constants import GCS_CASE_SOURCE
from gdmloader.source import SystemLoader

gdm_loader = SystemLoader()
gdm_loader.add_source(GCS_CASE_SOURCE)

catalog: DistributionSystem = gdm_loader.load_dataset(
    source_name=GCS_CASE_SOURCE.name, 
    system_type=DistributionSystem, 
    dataset_name="p1rhs7_1247",
)
catalog.name = "p1rhs7_1247"
catalog.info()

/opt/homebrew/Caskroom/miniconda/base/envs/gdm2/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: Ellipsis is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


System                                
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Property             ┃       Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ System name          │ p1rhs7_1247 │
│ Data format version  │       2.0.1 │
│ Components attached  │       13370 │
│ Time Series attached │           0 │
│ Description          │             │
└──────────────────────┴─────────────┘

Component Information                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Type                             ┃ Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ DistributionBus                  │  2510 │
│ DistributionLoad                 │  1896 │
│ DistributionTransformer          │   503 │
│ DistributionTransformerEquipment │    10 │
│ DistributionVoltageSource        │     1 │
│ LoadEquipment                    │  1896 │
│ Location                         │  2510 │
│ MatrixImpedanceBranch            │  1841 │
│ MatrixImpedanceBranchEquipment   │    38 │
│ MatrixImpedanceFuse              │    81 │
│ MatrixImpedanceFuseEquipment     │     6 │
│ MatrixImpedanceSwitch            │    84 │
│ MatrixImpedanceSwitchEquipment   │    15 │
│ PhaseLoadEquipment               │  1948 │
│ PhaseVoltageSourceEquipment      │     3 │
│ TimeCurrentCurve                 │     1 │
│ VoltageLimitSet                  │     8 │
│ VoltageSourceEquipment           │     1 │
│ WindingEquipment                 │    18 │
└──────────────────────────────────┴───────┘

SHIFT builds synthetic distribution models in the GDM format. These can then be converted to OpenDSS using DiTTo as needed. Models can be built from a polygon of interest, an address or from single line diagrams. The example below builds a gdm models for a given address.

In [3]:
from shift_interface import shift_model_builder, SourceSelection

synthetic_distribution_system = shift_model_builder(
    parcel_source="16550 Race Street, Thornton, CO, 80602",
    catalog_sys = catalog,
    radius_in_meters=1000,
    homes_per_distribution_secondary=4,
    source_placement=SourceSelection.CENTER,
    is_single_phase=True,
    primary_voltage_level_in_kilovolts=7.2,
    secondary_voltage_level_in_kilovolts=0.12,
    gdm_model_name="gdm_model" 
)
synthetic_distribution_system.info()

2025-05-14 20:30:59.646 | INFO     | shift.parcel:parcels_from_location:148 - Attempting to fetch parcels for 16550 Race Street, Thornton, CO, 80602
2025-05-14 20:30:59.707 | INFO     | shift.parcel:parcels_from_geodataframe:24 - Length of geodataframe: 66, CRS: epsg:4326
2025-05-14 20:30:59.709 | INFO     | shift.parcel:parcels_from_geodataframe:95 - Number of parcels: 66
2025-05-14 20:30:59.710 | INFO     | shift_interface:shift_model_builder:183 - Found 66 parcels
2025-05-14 20:30:59.739 | INFO     | shift_interface:shift_model_builder:186 - Created 16 clusters
2025-05-14 20:30:59.740 | INFO     | shift_interface:shift_model_builder:191 - Source location: GeoLocation(longitude=-104.96594396888045, latitude=39.993612395066414)
2025-05-14 20:30:59.744 | DEBUG    | shift.openstreet_roads:get_road_network:43 - Attempting to fecth road network for POLYGON ((-104.97554935483134 39.986908245168664, -104.95663284516866 39.986908245168664, -104.95663284516866 40.001680354831336, -104.9755493

System                              
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Property             ┃     Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ System name          │ gdm_model │
│ Data format version  │     2.0.1 │
│ Components attached  │       864 │
│ Time Series attached │         0 │
│ Description          │           │
└──────────────────────┴───────────┘

Component Information                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Type                             ┃ Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ DistributionBus                  │   218 │
│ DistributionLoad                 │    54 │
│ DistributionTransformer          │    16 │
│ DistributionTransformerEquipment │     1 │
│ DistributionVoltageSource        │     1 │
│ LoadEquipment                    │    54 │
│ Location                         │   202 │
│ MatrixImpedanceBranch            │   201 │
│ MatrixImpedanceBranchEquipment   │     3 │
│ PhaseLoadEquipment               │   108 │
│ PhaseVoltageSourceEquipment      │     3 │
│ VoltageSourceEquipment           │     1 │
│ WindingEquipment                 │     2 │
└──────────────────────────────────┴───────┘